In [2]:
import pandas as pd
from litellm import completion
import litellm
import os
# litellm.enable_json_schema_validation = True
# litellm.set_verbose = True
import json
from litellm import ModelResponse
from dotenv import load_dotenv
from judge import Judge
load_dotenv()


True

In [3]:
try:
    df = pd.read_csv("base accuracy 100 gpt4o.csv")
except FileNotFoundError:
    df = pd.read_csv("100_data_math500.csv")

In [9]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str



In [10]:
def process_math_problems(df, start_idx:int, end_idx:int):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        problem = df.iloc[idx]['problem']
        answer = df.iloc[idx]["answer"]
        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': problem,
            }],
            response_format=Answer
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        llm_answer = answer_obj.answer

        judge = Judge(model='gemini/gemini-2.0-flash')
        answer_correctness_obj = judge.prediction(query=df.iloc[idx]['problem'],answer1=answer,answer2=llm_answer)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = llm_answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [ ]:
process_math_problems(df,start_idx=0,end_idx=100)

In [12]:
df.to_csv("base accuracy 100 gpt4o.csv")

In [13]:
df['is_correct'].value_counts()

is_correct
True     64
False    36
Name: count, dtype: int64